In [51]:
# -*- coding: UTF-8 -*-

import pandas as pd
import numpy as np

# train_df = pd.read_csv('/Users/Rosemary/Downloads/data_format1/train_format1.csv', sep=',')
train_df = pd.read_csv('/Users/hanmufu/Workspaces/Repeat_Buyers_Prediction/data_format1/train_format1.csv', dtype=str)
print("train_format1 loaded successful")
print(train_df.head())

# user_log_df = pd.read_csv('/Users/Rosemary/Downloads/data_format1/user_log_format1.csv')
user_log_df = pd.read_csv('/Users/hanmufu/Workspaces/Repeat_Buyers_Prediction/data_format1/user_log_format1.csv', dtype=str)
print("user_log_format1 loaded successful")
print(user_log_df.head())

# user_info_df = pd.read_csv('/Users/Rosemary/Downloads/data_format1/user_info_format1.csv')
user_info_df = pd.read_csv('/Users/hanmufu/Workspaces/Repeat_Buyers_Prediction/data_format1/user_info_format1.csv', dtype=str)
print("user_info_format1 loaded successful")
print(user_info_df.head())

train_with_user_info = pd.merge(train_df, user_info_df)
print(train_with_user_info.head())

user_log_df['month'] = user_log_df['time_stamp'].str[:2]

train_format1 loaded successful
  user_id merchant_id label
0   34176        3906     0
1   34176         121     0
2   34176        4356     1
3   34176        2217     0
4  230784        4818     0
user_log_format1 loaded successful
  user_id  item_id cat_id seller_id brand_id time_stamp action_type
0  328862   323294    833      2882     2661       0829           0
1  328862   844400   1271      2882     2661       0829           0
2  328862   575153   1271      2882     2661       0829           0
3  328862   996875   1271      2882     2661       0829           0
4  328862  1086186   1271      1253     1049       0829           0
user_info_format1 loaded successful
  user_id age_range gender
0  376517         6      1
1  234512         5      0
2  344532         5      0
3  186135         5      0
4   30230         5      0
  user_id merchant_id label age_range gender
0   34176        3906     0         6      0
1   34176         121     0         6      0
2   34176        4356   

In [59]:
user_log_df = user_log_df.rename(columns={'seller_id': 'merchant_id'})

In [105]:
# 计算user和merchant产生多少次action 十一月和之前几个月的均值的差
user_actions_at_nov = user_log_df.loc[user_log_df['month'] == "11"]
user_actions_not_at_nov = user_log_df.loc[user_log_df['month'] != "11"]
user_actions_not_at_nov = user_actions_not_at_nov.groupby(['user_id', 'merchant_id', 'month'], as_index=False)['month'].agg({'cnt':'count'})
user_actions_not_at_nov['user_id/merchant_id'] = user_actions_not_at_nov['user_id'] + '/' + user_actions_not_at_nov['merchant_id']
user_actions_not_at_nov = user_actions_not_at_nov.pivot(index='user_id/merchant_id', columns='month', values='cnt')
user_actions_not_at_nov = user_actions_not_at_nov.fillna(0)
user_actions_not_at_nov['mean_of_num_of_actions_not_in_nov'] = user_actions_not_at_nov.mean(axis=1)
user_actions_not_at_nov['user_id/merchant_id'] = user_actions_not_at_nov.index
user_actions_not_at_nov['user_id'] = user_actions_not_at_nov['user_id/merchant_id'].str.split('/').str[0]
user_actions_not_at_nov['merchant_id'] = user_actions_not_at_nov['user_id/merchant_id'].str.split('/').str[1]
user_actions_at_nov = user_actions_at_nov.groupby(['user_id', 'merchant_id', 'month'], as_index=False)['month'].agg({'cnt':'count'})
res = pd.merge(user_actions_at_nov, user_actions_not_at_nov, on=['user_id', 'merchant_id'])
res['user_action_deviation_between_nov_and_before'] = abs(res['cnt'] - res['mean_of_num_of_actions_not_in_nov'])
train_df['user_action_deviation_between_nov_and_before'] = res['user_action_deviation_between_nov_and_before']


In [104]:
print(res.head())

  user_id merchant_id month  cnt   05   06   07   08   09    10  \
0     100           1    11   31  0.0  0.0  0.0  0.0  7.0  61.0   
1     100        1461    11    7  0.0  0.0  0.0  0.0  0.0   2.0   
2     100        1860    11    6  0.0  0.0  0.0  0.0  0.0   1.0   
3     100        2020    11    1  0.0  0.0  0.0  0.0  0.0   9.0   
4     100        2669    11    3  0.0  0.0  0.0  0.0  0.0   1.0   

   mean_of_num_of_actions_not_in_nov user_id/merchant_id  \
0                          11.333333               100/1   
1                           0.333333            100/1461   
2                           0.166667            100/1860   
3                           1.500000            100/2020   
4                           0.166667            100/2669   

   action_variant_between_nov_and_before  
0                              19.666667  
1                               6.666667  
2                               5.833333  
3                               0.500000  
4                    

In [ ]:
# name columns
user_actions_not_at_nov.columns = ['user_id', 'seller_id', 'month', 'may', 'jun', 'july', 'aug', 'sep', 'oct', 'nov']

In [ ]:
user_actions_not_at_nov['may', 'jun', 'july', 'aug', 'sep', 'oct', 'nov'].mean()